In [211]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import locale #mudar o local para portuges
import re

In [212]:
from platform import python_version
print("Versão da Linguagem Python usada neste Notebook", python_version())

Versão da Linguagem Python usada neste Notebook 3.11.5


In [213]:
df = pd.read_excel('../data/dataset.xlsx')

In [214]:
df.head(3)

,ID_Pedido,ID_Vendedor,ID_Cliente,Equipe_Entrega,Cliente,Canal_Entrega,ID_Cidade,Data_Pedido,Data_Entrega_Prevista,Data_Entrega_Realizada,Status_Entrega
0,5630436,3459,72805,Norte,Cliente 1120,Canal12,79,"quarta-feira, 18 de dezembro de 2019","domingo, 22 de dezembro de 2019","sexta-feira, 20 de dezembro de 2019",Antecipado
1,5629618,3459,130086,Norte,Cliente 2351,Canal12,79,"quarta-feira, 18 de dezembro de 2019","domingo, 22 de dezembro de 2019","sexta-feira, 20 de dezembro de 2019",Antecipado
2,5629847,3459,59873,Norte,Cliente 7995,Canal12,79,"quarta-feira, 18 de dezembro de 2019","domingo, 22 de dezembro de 2019","sexta-feira, 20 de dezembro de 2019",Antecipado


In [215]:
total_entrega = df['ID_Pedido'].count()
print(total_entrega)

53770


In [216]:
df.columns

Index(['ID_Pedido', 'ID_Vendedor', 'ID_Cliente', 'Equipe_Entrega', 'Cliente',
       'Canal_Entrega', 'ID_Cidade', 'Data_Pedido', 'Data_Entrega_Prevista',
       'Data_Entrega_Realizada', 'Status_Entrega'],
      dtype='object')

In [217]:
df.shape

(53770, 11)

In [218]:
df.dtypes

ID_Pedido                  int64
ID_Vendedor                int64
ID_Cliente                 int64
Equipe_Entrega            object
Cliente                   object
Canal_Entrega             object
ID_Cidade                  int64
Data_Pedido               object
Data_Entrega_Prevista     object
Data_Entrega_Realizada    object
Status_Entrega            object
dtype: object

In [219]:
# Aplica o filtro de entregas no prazo
entregas_no_prazo = df[df['Status_Entrega'] == 'No Prazo']

# Agrupa o total de entregas no prazo por Canal_Entrega
entregas_agrupadas = entregas_no_prazo.groupby('Canal_Entrega')

# Contando o total de entregas em cada grupo
entregas_por_canal = entregas_agrupadas['Status_Entrega'].value_counts()

# Transforma a série em um DataFrame e reseta o índice
df_entregas_por_canal = entregas_por_canal.reset_index(name='Total_Entregas_No_Prazo')

In [220]:
grafico1 = px.bar(
    df_entregas_por_canal,
    orientation='h',
    x='Total_Entregas_No_Prazo',
    y='Canal_Entrega',
    text='Total_Entregas_No_Prazo',
    category_orders={"Canal_Entrega": df_entregas_por_canal.sort_values(by='Total_Entregas_No_Prazo', ascending=False)["Canal_Entrega"].values.tolist()}, # Ordenar pelo DataFrame original
    title='Entregas no prazo Por Canal de Entrega'
)
grafico1.update_layout(width=1000, height=500)

In [221]:
total_entregas = df.shape[0]
entregas_no_antecipadas = df[df['Status_Entrega'] == 'Antecipado']

percentual_entrega_antecipadas = round(entregas_no_antecipadas.shape[0] / total_entregas * 100,2)
percentual_entrega_antecipadas #percentual total de entregas antecipadas
entrega_por_distribuidora = entregas_no_antecipadas.groupby('Equipe_Entrega').size()

entrega_por_distribuidora = entrega_por_distribuidora.reset_index(name='Entregas_Antecipadas')

entrega_por_distribuidora.sort_values(by='Entregas_Antecipadas', ascending=False).reset_index().drop(columns='index')
#percentual de cada distribuidora 
entrega_por_distribuidora['percentual'] = round(entrega_por_distribuidora['Entregas_Antecipadas'] /total_entregas * 100)
entrega_por_distribuidora

,Equipe_Entrega,Entregas_Antecipadas,percentual
0,Distribuidores,127,0.0
1,Internet,3202,6.0
2,Nordeste,6821,13.0
3,Norte,11104,21.0
4,Sudeste,10937,20.0
5,Sul,3583,7.0
6,Televendas,2195,4.0
7,Venda Direta,52,0.0


In [222]:
table = go.Figure(data=[go.Table(
    header=dict(values=list(entrega_por_distribuidora.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[entrega_por_distribuidora.Equipe_Entrega, entrega_por_distribuidora.Entregas_Antecipadas, entrega_por_distribuidora.percentual],
               fill_color='lavender',
               align='left'))
])
table.update_layout(
    width=500,
    height=len(entrega_por_distribuidora['Equipe_Entrega']) * 50,  # Defina um fator de escala adequado para ajustar a altura
    title={'text': 'Entregas Antecipadas Por Equipe de Entrega', 'x': 0.5, 'xanchor': 'center'}
)
table.show()

In [223]:
#Pegando somente o nome do mes
df['Mes_Entrega'] = df['Data_Entrega_Realizada'].apply(lambda x: re.search(r'de (\w+) de', x).group(1))
#funcao para pegar o numero do mes
mes_para_numero = {
    'janeiro':1,
    'fevereiro':2,
    'março':3,
    'abril':4,
    'maio':5,
    'junho':6,
    'julho':7,
    'agosto':8,
    'setembro':9,
    'outubro':10,
    'novembro':11,
    'dezembro':12
}
#aplicando a funcao e criando a coluna mes numerico
df['mes_numerico'] = df['Mes_Entrega'].apply(lambda x: mes_para_numero[x])
df

,ID_Pedido,ID_Vendedor,ID_Cliente,Equipe_Entrega,Cliente,Canal_Entrega,ID_Cidade,Data_Pedido,Data_Entrega_Prevista,Data_Entrega_Realizada,Status_Entrega,Mes_Entrega,mes_numerico
0,5630436,3459,72805,Norte,Cliente 1120,Canal12,79,"quarta-feira, 18 de dezembro de 2019","domingo, 22 de dezembro de 2019","sexta-feira, 20 de dezembro de 2019",Antecipado,dezembro,12
1,5629618,3459,130086,Norte,Cliente 2351,Canal12,79,"quarta-feira, 18 de dezembro de 2019","domingo, 22 de dezembro de 2019","sexta-feira, 20 de dezembro de 2019",Antecipado,dezembro,12
2,5629847,3459,59873,Norte,Cliente 7995,Canal12,79,"quarta-feira, 18 de dezembro de 2019","domingo, 22 de dezembro de 2019","sexta-feira, 20 de dezembro de 2019",Antecipado,dezembro,12
3,5630482,3459,76035,Norte,Cliente 2236,Canal12,79,"quarta-feira, 18 de dezembro de 2019","domingo, 22 de dezembro de 2019","sexta-feira, 20 de dezembro de 2019",Antecipado,dezembro,12
4,5628297,3459,72610,Norte,Cliente 3081,Canal12,79,"terça-feira, 17 de dezembro de 2019","sábado, 21 de dezembro de 2019","quinta-feira, 19 de dezembro de 2019",Antecipado,dezembro,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53765,1341717,3802,77360,Norte,Cliente 10395,Canal08,70,"segunda-feira, 15 de julho de 2019","sexta-feira, 19 de julho de 2019","quarta-feira, 24 de julho de 2019",Atrasado,julho,7
53766,5519371,3827,171741,Norte,Cliente 7792,Canal17,16,"quarta-feira, 10 de julho de 2019","domingo, 14 de julho de 2019","sexta-feira, 19 de julho de 2019",Atrasado,julho,7
53767,5471229,3342,144698,Sul,Cliente 5161,Canal07,84,"quarta-feira, 17 de abril de 2019","domingo, 21 de abril de 2019","sexta-feira, 26 de abril de 2019",Atrasado,abril,4
53768,5471072,3342,170914,Sul,Cliente 69,Canal03,84,"quarta-feira, 17 de abril de 2019","domingo, 21 de abril de 2019","sexta-feira, 26 de abril de 2019",Atrasado,abril,4


In [224]:
df_meses = df[['Mes_Entrega']]
df_total_mes = df_meses.groupby('Mes_Entrega').size().reset_index()
df_total_mes.columns = ['Mes_Entrega', 'Total_Entrega']
df_total_mes['Mes_Numero'] = df_total_mes['Mes_Entrega'].apply(lambda x: mes_para_numero[x])
df_total_mes = df_total_mes.sort_values('Mes_Numero')

In [225]:
fig = go.Figure(data=go.Scatter(x=df_total_mes['Mes_Entrega'], y=df_total_mes['Total_Entrega'], mode='lines+markers'))

# Configurando o layout do gráfico
fig.update_layout(title='Total de Entregas por Mês',
                  xaxis_title='Mês',
                  yaxis_title='Total de Entregas',
                  width=960, height=400)

# Exibindo o gráfico
fig.show()

In [226]:
df_vendedor = df.groupby('ID_Vendedor').size().reset_index()
df_vendedor.columns = ['ID_Vendedor', 'Total_Venda']
df_vendedor = df_vendedor.sort_values(by='Total_Venda', ascending=False)
df_vendedor = df_vendedor.head(5)
df_vendedor['ID_Vendedor'] = df_vendedor['ID_Vendedor'].apply(lambda x: f'Vendedor_{x}')
df_vendedor

,ID_Vendedor,Total_Venda
63,Vendedor_3894,2208
38,Vendedor_3686,1936
3,Vendedor_201,1837
24,Vendedor_3459,1771
44,Vendedor_3772,1676


In [227]:
grafico4 = px.bar(
    df_vendedor,
    orientation='h',
    x='Total_Venda',
    y='ID_Vendedor',
    text='Total_Venda',
    category_orders={"ID_Vendedor": df_vendedor.sort_values(by='Total_Venda', ascending=False)["ID_Vendedor"].values.tolist()}  # Ordenar pelo DataFrame original
)
grafico4.update_layout(width=500, height=600, title={'text': 'Top 5 Vendedores', 'x': 0.5, 'xanchor': 'center'})
grafico4.show()

In [228]:
df_atraso = df[df['Status_Entrega'] == 'Atrasado']
df_atraso = df_atraso.groupby('ID_Cidade').size()
df_atraso = df_atraso.reset_index()
df_atraso.columns = ['ID_Cidade', 'Total_Entrega_Atrasado']
df_atraso
df_atraso['ID_Cidade'] = df_atraso['ID_Cidade'].astype(str)
df_atraso = df_atraso.sort_values(by='Total_Entrega_Atrasado', ascending=False)
# total_entregas_cidade = df_atraso.groupby('ID_Cidade')

In [229]:
table2 = go.Figure(data=[go.Table(
    header=dict(values=list(df_atraso.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_atraso.ID_Cidade, df_atraso.Total_Entrega_Atrasado],
               fill_color='lavender',
               align='left'))
])
table2.update_layout(width=800, height=400)

table2.show()

In [230]:
df_entregas = df['Status_Entrega'].value_counts().reset_index()
df_entregas.columns = ['Status_Entrega', 'Total_Entrega']
df_entregas['Percentual_Entregas'] = round(df_entregas['Total_Entrega'] / total_entregas * 100, 2)
df_entregas


,Status_Entrega,Total_Entrega,Percentual_Entregas
0,Antecipado,38021,70.71
1,No Prazo,8772,16.31
2,Atrasado,6977,12.98


In [231]:
grafico6 = px.pie(data_frame=df_entregas,
                  values='Total_Entrega',
                  names='Status_Entrega',
                  title='Percentual de Entregas Por Status de Entrega'
                  )
grafico6.update_traces(textposition='inside', textinfo='percent+label')
grafico6.update_layout(width=600, height=500, title={'text': 'Percentual de Entregas Por Status de Entrega', 'x': 0.5, 'xanchor': 'center'})
grafico6.show()

In [232]:
df.head()

,ID_Pedido,ID_Vendedor,ID_Cliente,Equipe_Entrega,Cliente,Canal_Entrega,ID_Cidade,Data_Pedido,Data_Entrega_Prevista,Data_Entrega_Realizada,Status_Entrega,Mes_Entrega,mes_numerico
0,5630436,3459,72805,Norte,Cliente 1120,Canal12,79,"quarta-feira, 18 de dezembro de 2019","domingo, 22 de dezembro de 2019","sexta-feira, 20 de dezembro de 2019",Antecipado,dezembro,12
1,5629618,3459,130086,Norte,Cliente 2351,Canal12,79,"quarta-feira, 18 de dezembro de 2019","domingo, 22 de dezembro de 2019","sexta-feira, 20 de dezembro de 2019",Antecipado,dezembro,12
2,5629847,3459,59873,Norte,Cliente 7995,Canal12,79,"quarta-feira, 18 de dezembro de 2019","domingo, 22 de dezembro de 2019","sexta-feira, 20 de dezembro de 2019",Antecipado,dezembro,12
3,5630482,3459,76035,Norte,Cliente 2236,Canal12,79,"quarta-feira, 18 de dezembro de 2019","domingo, 22 de dezembro de 2019","sexta-feira, 20 de dezembro de 2019",Antecipado,dezembro,12
4,5628297,3459,72610,Norte,Cliente 3081,Canal12,79,"terça-feira, 17 de dezembro de 2019","sábado, 21 de dezembro de 2019","quinta-feira, 19 de dezembro de 2019",Antecipado,dezembro,12


In [233]:
df.to_excel('../data/datasetTratado.xlsx', index=False)